In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras import backend as K



Using TensorFlow backend.


In [3]:
## Load data into dataframes

import pandas as pd
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [4]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
## split train data into train and cv by stratified sampling

from sklearn.model_selection import train_test_split

Y = train['label']
xtrain, xcv, ytrain, ycv = train_test_split(train, Y, test_size=0.33, stratify=Y)

In [7]:
xtrain=xtrain[xtrain.columns[1:785]]
xcv=xcv[xcv.columns[1:785]]

In [8]:
print(xtrain.shape, ytrain.shape)
print(xcv.shape, ycv.shape)

(28140, 784) (28140,)
(13860, 784) (13860,)


In [9]:
xtrainn=xtrain.to_numpy()
xcvv=xcv.to_numpy()
ytrainn=ytrain.to_numpy()
ycvv=ycv.to_numpy()

In [10]:
xtestt=test.to_numpy()

In [11]:
## Convert class labels to vectors
from keras.utils import np_utils
ytrainn=np_utils.to_categorical(ytrainn, 10)
ycvv=np_utils.to_categorical(ycvv, 10)

In [12]:
print(ytrainn[3])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [13]:
batchsize=128
numclasses=10
epochs=12

## input image dimensions
imgrows, imgcols=28,28

In [14]:
if K.image_data_format=='channels_first':
    xtrainn=xtrainn.reshape(xtrainn.shape[0], 1, imgrows, imgcols)
    xcvv=xcvv.reshape(xcvv.shape[0], 1, imgrows, imgcols)
    inputshape=(1, imgrows, imgcols)
else:
    xtrainn=xtrainn.reshape(xtrainn.shape[0], imgrows, imgcols, 1)
    xcvv=xcvv.reshape(xcvv.shape[0], imgrows, imgcols, 1)
    inputshape=(imgrows, imgcols, 1)

In [15]:
## Normalize data

xtrainn=xtrainn/255
xcvv=xcvv/255

In [16]:
print(xtrainn.shape, ytrainn.shape)
print(xcvv.shape, ycvv.shape)

(28140, 28, 28, 1) (28140, 10)
(13860, 28, 28, 1) (13860, 10)


## **Model with Kernel size 3X3**

In [17]:
## kernel size 3X3, Maxpooling, dropouts 0.3, 0.5

model1=Sequential()
model1.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=inputshape))
model1.add(Conv2D(64, (3,3), activation='relu'))
model1.add(Conv2D(128, (3,3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.25))

model1.add(Dense(numclasses, activation='softmax'))

model1.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

model1.fit(xtrainn, ytrainn, batch_size=batchsize, epochs=epochs, verbose=1, 
          validation_data=(xcvv, ycvv))

score1=model1.evaluate(xcvv, ycvv, verbose=0)
print("Validation Loss= ", score1[0])
print("Validation Accuracy= ",score1[1])

Train on 28140 samples, validate on 13860 samples
Epoch 1/12
28140/28140 [==============================] - 145s 5ms/step - loss: 0.3320 - accuracy: 0.8979 - val_loss: 0.0749 - val_accuracy: 0.9774
Epoch 2/12
28140/28140 [==============================] - 145s 5ms/step - loss: 0.0774 - accuracy: 0.9756 - val_loss: 0.0852 - val_accuracy: 0.9740
Epoch 3/12
28140/28140 [==============================] - 144s 5ms/step - loss: 0.0521 - accuracy: 0.9842 - val_loss: 0.0484 - val_accuracy: 0.9846
Epoch 4/12
28140/28140 [==============================] - 144s 5ms/step - loss: 0.0384 - accuracy: 0.9882 - val_loss: 0.0441 - val_accuracy: 0.9868
Epoch 5/12
28140/28140 [==============================] - 145s 5ms/step - loss: 0.0287 - accuracy: 0.9903 - val_loss: 0.0435 - val_accuracy: 0.9872
Epoch 6/12
28140/28140 [==============================] - 144s 5ms/step - loss: 0.0227 - accuracy: 0.9924 - val_loss: 0.0438 - val_accuracy: 0.9877
Epoch 7/12
28140/28140 [==============================] - 146s

In [18]:

## Test class label predictions
if K.image_data_format=='channels_first':
    xtestt=xtestt.reshape(xtestt.shape[0], 1, imgrows, imgcols)
    
else:
    xtestt=xtestt.reshape(xtestt.shape[0], imgrows, imgcols, 1)
print(xtestt.shape)
ytest=model1.predict_classes(xtestt)

(28000, 28, 28, 1)


In [19]:
ids=np.arange(1,28001)
print(ids)
print(ytest)
results_cnn=pd.DataFrame({'ImageId':ids, 'Label':ytest})
results_cnn.head()

[    1     2     3 ... 27998 27999 28000]
[2 0 9 ... 3 9 2]


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [20]:
filename='Digit_Recognizer_Predictions_CNN.csv'
results_cnn.to_csv(filename, index=False)
print('Saved file ',filename)

Saved file  Digit_Recognizer_Predictions_CNN.csv
